In [1]:
from selenium import webdriver

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.expected_conditions import visibility_of_element_located
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.wait import WebDriverWait
from datetime import datetime

import pandas as pd
import numpy as np
import time
import random
import os
import re

In [2]:
now = datetime.now()

website = "medium"

start_date = now.strftime('%m%d%Y')
# make data/playlist/ directory
os.makedirs(website + '/' + start_date , exist_ok=True)

driver = webdriver.Chrome(ChromeDriverManager().install())
chrome_options = Options()
chrome_options.add_argument("--start-maximized")

url = r'http://medium.com/'

driver.get(url)
driver.implicitly_wait(8)
# Log in

[WDM] - Downloading: 100%|██████████| 6.46M/6.46M [00:00<00:00, 10.1MB/s]
C:\Users\user\AppData\Local\Temp\ipykernel_5604\3494290274.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [15]:
driver.maximize_window()
keyword = 'Data Science'

# best_of (options):  This week , This month , This year , 'All time

best_of  = 'This month'

os.makedirs(website + '/' + start_date + '/' + best_of, exist_ok=True)

search_field = driver.find_element(By.XPATH, '//input[@placeholder="Search"]')
search_field.clear()
search_field.send_keys(keyword)
search_field.send_keys(Keys.ENTER)

keyword_tag = WebDriverWait(driver, 20).until(
                    EC.element_to_be_clickable(driver.find_element(By.XPATH, f"//div[normalize-space()='{keyword}']")))
keyword_tag.click()

time.sleep(3)

best_a = WebDriverWait(driver, 20).until(
                    EC.element_to_be_clickable(driver.find_element(By.XPATH, f"//a[normalize-space()='Best']")))
best_a.click()
drop_down = WebDriverWait(driver, 20).until(
                    EC.element_to_be_clickable(driver.find_element(By.XPATH, "//span[normalize-space()='This year']")))
drop_down.click()
all_time = WebDriverWait(driver, 20).until(
                    EC.element_to_be_clickable(driver.find_element(By.XPATH, f"//button[normalize-space()='{best_of}']")))
all_time.click()

allowance = 40
target = 200
scroll_speed = 20

target_articles = target + allowance
scroll_count = 0

while True:
    scroll_count += scroll_speed
    driver.execute_script('window.scrollTo(0, {});'.format(scroll_count))

    article_count =  int(driver.execute_script("return document.getElementsByTagName('article').length"))

    if article_count >= target_articles:
        print(f"number of articles found: {article_count}")
        break

# Get links within articles
article_urls = []

articles = driver.find_elements(By.TAG_NAME, "article")

for article in articles:
    article_url = article.find_element(By.TAG_NAME, "h2").find_element(By.XPATH, "..").find_element(By.XPATH, "..").get_attribute('href')
    article_urls.append(article_url)


number of articles found: 245


In [16]:
driver.maximize_window()
link_num = 0
adjustment = 0

for article_url in article_urls:

    initial_scroll = 50
    add_scroll = 0
    link_num += 1

    try:
        driver.get(article_url)

        page_num = link_num - adjustment

        print(f"Start! link: {link_num}, page: {page_num}, adjustment:{adjustment}")

        driver.execute_script(f'window.scrollTo(0, {initial_scroll + add_scroll})')

        button_session = 0
        fails = 0
        while button_session < 1:
            try:
                if fails == 3:
                    break

                claps_button = WebDriverWait(driver, 20).until(
                                    EC.element_to_be_clickable(
                                        driver.find_element(By.CLASS_NAME, "pw-multi-vote-count").find_element(By.TAG_NAME, 'button')
                                        ))
                claps_button.click()
                button_session += 1
            except:
                fails += 1
                add_scroll += 10
                driver.execute_script(f'window.scrollTo(0, {initial_scroll + add_scroll})')

                try:
                    claps_icon = WebDriverWait(driver, 20).until(
                            EC.element_to_be_clickable(
                                driver.find_element(By.CLASS_NAME, "pw-multi-vote-icon").find_element(By.TAG_NAME, 'button')
                                ))
                    claps_icon.click()

                except:
                    time.sleep(1) 

                time.sleep(2)
        try:
            claps_text = driver.find_element(By.XPATH, "//h2[contains(text(), 'claps from')]").text

        except:
            try:
                claps_text = driver.find_element(By.XPATH, "//h2[contains(text(), 'clap from')]").text
            except:
                claps_text = ""

        pattern = r"(\d{1,9})"

        clap_count = int(re.findall(pattern, claps_text)[0])

        unique_clap_count = int(re.findall(pattern, claps_text)[1])
        time.sleep(2)
        
        try:
            close_claps = WebDriverWait(driver, 20).until(
                                EC.element_to_be_clickable(driver.find_element(By.XPATH, "//button[@aria-label='close']")))

            close_claps.click()
        except:
            print('close button failed')

        title = driver.find_element(By.CLASS_NAME, "pw-post-title").text

        summary = driver.find_element(By.TAG_NAME, 'section').text.strip(" ")

        image_count = int(driver.execute_script("return document.getElementsByTagName('section')[0].getElementsByTagName('img').length"))

        link_count = int(
            driver.execute_script(
                "return document.getElementsByTagName('section')[0].getElementsByTagName('a').length"))

        blockquote_count = int(driver.execute_script("return document.getElementsByTagName('blockquote').length"))

        publication_date = driver.find_element(By.CLASS_NAME, "pw-published-date").text

        reading_time_text = driver.find_element(By.CLASS_NAME, "pw-reading-time").text

        reading_time = int(re.findall(pattern, reading_time_text)[0])

        try:
            comment_count = int(driver.find_element(By.CLASS_NAME, "pw-responses-count").text)

        except:
            comment_count = 0

        author_url = driver.find_element(By.CLASS_NAME, "pw-author-name").find_element(By.XPATH, "..").get_attribute('href')

        author_name = driver.find_element(By.CLASS_NAME, "pw-author-name").text.strip(" ")

        try:
            publication_url = driver.find_element(By.XPATH, "//div[normalize-space()='Published in']").find_element(By.XPATH, "../..").find_elements(By.TAG_NAME, 'a')[1].get_attribute('href')
            publication_name = driver.find_element(By.XPATH, "//div[normalize-space()='Published in']").find_element(By.XPATH, "../..").find_elements(By.TAG_NAME, 'a')[1].find_element(By.TAG_NAME, "p").text.strip(" ")

        except:
            publication_url = None
            publication_name = None

        try:
            codeblock_count = int(
                driver.execute_script(
                    "return document.getElementsByTagName('pre').length"))
            br_count = 0
            code_blocks = driver.find_elements(By.TAG_NAME,  "pre")
            for code_block in code_blocks:
                brakes_found = len(code_block.find_elements(By.TAG_NAME, 'br'))
                br_count += brakes_found

            code_count = br_count + codeblock_count
        except:
            codeblock_count = 0
            code_count = 0

        # all_codes_list = []

        # for code_block in code_blocks:
        #     codes_found = code_block.text.split('\n')
        #     all_codes_list.extend(codes_found)

        article_info = {
            "article_url" : article_url, # change later
            "best_of": best_of,
            "title": title,
            "summary": summary,
            "image_count": image_count,
            "link_count": link_count,
            "blockquote_count": blockquote_count,
            "publication_date": publication_date,
            'reading_time': reading_time,
            "clap_count" : clap_count,
            "unique_clap_count": unique_clap_count,
            "comment_count": comment_count,
            "author_url": author_url,
            "author_name": author_name,
            "publication_url": publication_url,
            "publication_name": publication_name,
            "codeblock_count": codeblock_count,
            "code_count": code_count,
            # "all_codes_list": all_codes_list
        }

        page_data = pd.DataFrame([article_info])

        page_data.to_csv(website + '/' + start_date + '/' + best_of  + '/page' + str(page_num)+ ".csv", index=False )

        print(f"Data saved for link: {link_num}, page: {page_num}")

        if page_num >= target: # can change to target later
            break 

    except:
        adjustment += 1
        continue

print(f"""
Scraping Completed. 
total pages: {page_num}, 
total adjustments: {adjustment}
""")

Start! link: 1, page: 1, adjustment:0
Data saved for link: 1, page: 1
Start! link: 2, page: 2, adjustment:0
Data saved for link: 2, page: 2
Start! link: 3, page: 3, adjustment:0
Data saved for link: 3, page: 3
Start! link: 4, page: 4, adjustment:0
Data saved for link: 4, page: 4
Start! link: 5, page: 5, adjustment:0
Data saved for link: 5, page: 5
Start! link: 6, page: 6, adjustment:0
Data saved for link: 6, page: 6
Start! link: 7, page: 7, adjustment:0
Data saved for link: 7, page: 7
Start! link: 8, page: 8, adjustment:0
Data saved for link: 8, page: 8
Start! link: 9, page: 9, adjustment:0
Data saved for link: 9, page: 9
Start! link: 10, page: 10, adjustment:0
Data saved for link: 10, page: 10
Start! link: 11, page: 11, adjustment:0
Data saved for link: 11, page: 11
Start! link: 12, page: 12, adjustment:0
Data saved for link: 12, page: 12
Start! link: 13, page: 13, adjustment:0
Data saved for link: 13, page: 13
Start! link: 14, page: 14, adjustment:0
Data saved for link: 14, page: 14


In [17]:
driver.maximize_window()
link_num = 167
adjustment = 10

for article_url in article_urls:

    initial_scroll = 50
    add_scroll = 0
    link_num += 1

    try:
        driver.get(article_url)

        page_num = link_num - adjustment 

        print(f"Start! link: {link_num}, page: {page_num}, adjustment:{adjustment}")

        driver.execute_script(f'window.scrollTo(0, {initial_scroll + add_scroll})')

        button_session = 0
        fails = 0
        while button_session < 1:
            try:
                if fails == 3:
                    break

                claps_button = WebDriverWait(driver, 20).until(
                                    EC.element_to_be_clickable(
                                        driver.find_element(By.CLASS_NAME, "pw-multi-vote-count").find_element(By.TAG_NAME, 'button')
                                        ))
                claps_button.click()
                button_session += 1
            except:
                fails += 1
                add_scroll += 10
                driver.execute_script(f'window.scrollTo(0, {initial_scroll + add_scroll})')

                try:
                    claps_icon = WebDriverWait(driver, 20).until(
                            EC.element_to_be_clickable(
                                driver.find_element(By.CLASS_NAME, "pw-multi-vote-icon").find_element(By.TAG_NAME, 'button')
                                ))
                    claps_icon.click()

                except:
                    time.sleep(1) 

                time.sleep(2)
        try:
            claps_text = driver.find_element(By.XPATH, "//h2[contains(text(), 'claps from')]").text

        except:
            try:
                claps_text = driver.find_element(By.XPATH, "//h2[contains(text(), 'clap from')]").text
            except:
                claps_text = ""

        pattern = r"(\d{1,9})"

        clap_count = int(re.findall(pattern, claps_text)[0])

        unique_clap_count = int(re.findall(pattern, claps_text)[1])
        time.sleep(2)
        
        try:
            close_claps = WebDriverWait(driver, 20).until(
                                EC.element_to_be_clickable(driver.find_element(By.XPATH, "//button[@aria-label='close']")))

            close_claps.click()
        except:
            print('close button failed')

        title = driver.find_element(By.CLASS_NAME, "pw-post-title").text

        summary = driver.find_element(By.TAG_NAME, 'section').text.strip(" ")

        image_count = int(driver.execute_script("return document.getElementsByTagName('section')[0].getElementsByTagName('img').length"))

        link_count = int(
            driver.execute_script(
                "return document.getElementsByTagName('section')[0].getElementsByTagName('a').length"))

        blockquote_count = int(driver.execute_script("return document.getElementsByTagName('blockquote').length"))

        publication_date = driver.find_element(By.CLASS_NAME, "pw-published-date").text

        reading_time_text = driver.find_element(By.CLASS_NAME, "pw-reading-time").text

        reading_time = int(re.findall(pattern, reading_time_text)[0])

        try:
            comment_count = int(driver.find_element(By.CLASS_NAME, "pw-responses-count").text)

        except:
            comment_count = 0

        author_url = driver.find_element(By.CLASS_NAME, "pw-author-name").find_element(By.XPATH, "..").get_attribute('href')

        author_name = driver.find_element(By.CLASS_NAME, "pw-author-name").text.strip(" ")

        try:
            publication_url = driver.find_element(By.XPATH, "//div[normalize-space()='Published in']").find_element(By.XPATH, "../..").find_elements(By.TAG_NAME, 'a')[1].get_attribute('href')
            publication_name = driver.find_element(By.XPATH, "//div[normalize-space()='Published in']").find_element(By.XPATH, "../..").find_elements(By.TAG_NAME, 'a')[1].find_element(By.TAG_NAME, "p").text.strip(" ")

        except:
            publication_url = None
            publication_name = None

        try:
            codeblock_count = int(
                driver.execute_script(
                    "return document.getElementsByTagName('pre').length"))
            br_count = 0
            code_blocks = driver.find_elements(By.TAG_NAME,  "pre")
            for code_block in code_blocks:
                brakes_found = len(code_block.find_elements(By.TAG_NAME, 'br'))
                br_count += brakes_found

            code_count = br_count + codeblock_count
        except:
            codeblock_count = 0
            code_count = 0

        # all_codes_list = []

        # for code_block in code_blocks:
        #     codes_found = code_block.text.split('\n')
        #     all_codes_list.extend(codes_found)

        article_info = {
            "article_url" : article_url, # change later
            "best_of": best_of,
            "title": title,
            "summary": summary,
            "image_count": image_count,
            "link_count": link_count,
            "blockquote_count": blockquote_count,
            "publication_date": publication_date,
            'reading_time': reading_time,
            "clap_count" : clap_count,
            "unique_clap_count": unique_clap_count,
            "comment_count": comment_count,
            "author_url": author_url,
            "author_name": author_name,
            "publication_url": publication_url,
            "publication_name": publication_name,
            "codeblock_count": codeblock_count,
            "code_count": code_count,
            # "all_codes_list": all_codes_list
        }

        page_data = pd.DataFrame([article_info])

        page_data.to_csv(website + '/' + start_date + '/' + best_of  + '/page' + str(page_num)+ ".csv", index=False )

        print(f"Data saved for link: {link_num}, page: {page_num}")

        if page_num >= target: # can change to target later
            break 

    except:
        adjustment += 1
        continue

print(f"""
Scraping Completed. 
total pages: {page_num}, 
total adjustments: {adjustment}
""")

Start! link: 168, page: 158, adjustment:10
Data saved for link: 168, page: 158
Start! link: 169, page: 159, adjustment:10
Data saved for link: 169, page: 159
Start! link: 170, page: 160, adjustment:10
Data saved for link: 170, page: 160
Start! link: 171, page: 161, adjustment:10
Data saved for link: 171, page: 161
Start! link: 172, page: 162, adjustment:10
Data saved for link: 172, page: 162
Start! link: 173, page: 163, adjustment:10
Data saved for link: 173, page: 163
Start! link: 174, page: 164, adjustment:10
Data saved for link: 174, page: 164
Start! link: 175, page: 165, adjustment:10
Data saved for link: 175, page: 165
Start! link: 176, page: 166, adjustment:10
Data saved for link: 176, page: 166
Start! link: 177, page: 167, adjustment:10
Data saved for link: 177, page: 167
Start! link: 178, page: 168, adjustment:10
Data saved for link: 178, page: 168
Start! link: 179, page: 169, adjustment:10
Data saved for link: 179, page: 169
Start! link: 180, page: 170, adjustment:10
Data save

In [18]:
file_count = len(os.listdir( "./" + website + "/" + start_date + "/" + best_of))

result_df = pd.read_csv( "./" + website + "/" + start_date + "/" + best_of + "/page1.csv")

for page in range(2,file_count+1,1):

    print(f"loading page {page}....", end="")

    df_to_add = pd.read_csv( website + '/' + start_date + '/' + best_of + '/page'+ str(page)+ ".csv")

    result_df = pd.concat([result_df, df_to_add])

    print('Done!')

# page_data.to_csv(website + '/' + start_date + '/' + best_of  + '/page' + str(page_num)+ ".csv", index=False )

result_df.to_csv( website + '/top_' + str(file_count) + '_' + best_of + '_raw' + '.csv', index=False)

result_df.shape

loading page 2....Done!
loading page 3....Done!
loading page 4....Done!
loading page 5....Done!
loading page 6....Done!
loading page 7....Done!
loading page 8....Done!
loading page 9....Done!
loading page 10....Done!
loading page 11....Done!
loading page 12....Done!
loading page 13....Done!
loading page 14....Done!
loading page 15....Done!
loading page 16....Done!
loading page 17....Done!
loading page 18....Done!
loading page 19....Done!
loading page 20....Done!
loading page 21....Done!
loading page 22....Done!
loading page 23....Done!
loading page 24....Done!
loading page 25....Done!
loading page 26....Done!
loading page 27....Done!
loading page 28....Done!
loading page 29....Done!
loading page 30....Done!
loading page 31....Done!
loading page 32....Done!
loading page 33....Done!
loading page 34....Done!
loading page 35....Done!
loading page 36....Done!
loading page 37....Done!
loading page 38....Done!
loading page 39....Done!
loading page 40....Done!
loading page 41....Done!
loading 

(200, 18)

In [19]:
top200_df = pd.read_csv( website + '/top_' + str(file_count) + '_' + best_of + '_raw' + '.csv')
top200_df

,article_url,best_of,title,summary,image_count,link_count,blockquote_count,publication_date,reading_time,clap_count,unique_clap_count,comment_count,author_url,author_name,publication_url,publication_name,codeblock_count,code_count
0,https://medium.com/towards-data-science/meet-j...,This month,Meet Julia: The Future of Data Science,Meet Julia: The Future of Data Science\nThe ne...,7,30,0,Nov 2,7,1798,518,38,https://natassha6789.medium.com/?source=---thr...,Natassha Selvaraj,https://towardsdatascience.com/?source=post_pa...,Towards Data Science,1,2
1,https://medium.com/towards-data-science/the-op...,This month,The Open-Source Spirit of Data Science,The Open-Source Spirit of Data Science\nIt too...,1,22,0,Nov 10,3,35,12,3,https://towardsdatascience.medium.com/?source=...,TDS Editors,https://towardsdatascience.com/?source=post_pa...,Towards Data Science,0,0
2,https://medium.com/towards-data-science/how-da...,This month,How Data Scientists Level Up Their Coding Skills,How Data Scientists Level Up Their Coding Skil...,1,21,0,Oct 27,3,95,36,2,https://towardsdatascience.medium.com/?source=...,TDS Editors,https://towardsdatascience.com/?source=post_pa...,Towards Data Science,0,0
3,https://medium.com/towards-data-science/data-s...,This month,"Data Science Soft Skills, Revisited","Data Science Soft Skills, Revisited\nIf you lo...",1,25,0,Nov 3,3,134,18,0,https://towardsdatascience.medium.com/?source=...,TDS Editors,https://towardsdatascience.com/?source=post_pa...,Towards Data Science,0,0
4,https://medium.com/towards-data-science/discov...,This month,Discover New and Exciting Voices on TDS,Discover New and Exciting Voices on TDS\nFew t...,1,24,0,Oct 20,3,20,10,1,https://towardsdatascience.medium.com/?source=...,TDS Editors,https://towardsdatascience.com/?source=post_pa...,Towards Data Science,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://medium.com/geekculture/5-tools-that-ma...,This month,5 Tools That Make My Life Easier When Writing ...,5 Tools That Make My Life Easier When Writing ...,8,9,0,Oct 30,5,259,79,4,https://medium.com/@frank-andrade?source=---th...,Frank Andrade,https://medium.com/geekculture?source=post_pag...,Geek Culture,2,5
196,https://medium.com/towards-data-science/life-l...,This month,Life Lessons I Learned from Working as a Data ...,"Teluk Hijau, Banyuwangi (Image by Author)\nLif...",6,13,5,Oct 25,7,84,39,0,https://tanuwidjajaolivia.medium.com/?source=-...,Olivia Tanuwidjaja,https://towardsdatascience.com/?source=post_pa...,Towards Data Science,0,0
197,https://medium.com/@joulee/the-data-informed-m...,This month,Manifesto for the Data-Informed,Manifesto for the Data-Informed\nWhy is the pr...,1,2,0,Nov 4,5,957,126,10,https://joulee.medium.com/?source=---three_col...,Julie Zhuo,NaN,NaN,0,0
198,https://medium.com/codex/dont-use-iterrows-for...,This month,Follow This Approach to run 31x FASTER loops i...,Follow This Approach to run 31x FASTER loops i...,4,5,1,Oct 26,5,303,82,9,https://medium.com/@anmol3015?source=---three_...,Anmol Tomar,https://medium.com/codex?source=post_page-----...,CodeX,6,55
